In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import project specific functions
from column_map import column_map
from yesno_functions import *
from format_datetime import *

In [3]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [4]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [5]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [6]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('ambassador_cw')

In [7]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [8]:
cols = list(col_rename.values())

### Read the site reference list from SQL

In [9]:
sql = 'select * from crowdsdb.dbo.tbl_ref_sites'

In [10]:
site_ref = pd.read_sql(con = engine, sql = sql)[['site_id', 'site_desc', 'borough']]

### Read the current data from SQL

In [11]:
sql = 'select * from crowdsdb.dbo.tbl_cw_ambassador'

In [12]:
ambass_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['ambassador_id', 'patroncount'])
              .fillna(value = np.nan, axis = 1))

In [13]:
format_datetime(ambass_sql, 'encounter_timestamp')
format_datetime(ambass_sql, 'encounter_datetime')

In [14]:
sql_cols = list(ambass_sql.columns.values)

In [15]:
float_cols = ['sd_pdcontact', 'closed_approach', 'closed_outcome', 'closed_pdcontact']
for c in float_cols:
    ambass_sql[c] = ambass_sql[c].astype(float)

In [16]:
hash_rows(ambass_sql, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Read the latest data from Google Sheets

In [17]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
client = gspread.authorize(creds)

In [18]:
sheet = client.open('_COMBINED Inter-Agency Parks Social Distancing Education Ambassador Report (Responses)')

In [19]:
ws = sheet.worksheet('RESPONSES')

In [20]:
ambass = (get_as_dataframe(ws, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [21]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
1,6/5/2020 19:03:54,6/5/2020 16:23:00,Prospect Park,NaN,OATH,Alex,Kolodka,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn
2,6/5/2020 17:51:16,6/5/2020 17:30:00,Kissena Park,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,Open field/multi-purpose play area,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
3,6/5/2020 17:30:34,6/5/2020 17:30:00,Kissena Park,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,Open field/multi-purpose play area,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
4,6/5/2020 17:29:05,6/5/2020 14:45:00,Kissena Park,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
5,6/5/2020 17:03:29,5/5/2020 16:00:00,Flushing Fields,NaN,OATH,Emmanuelasia,Vaught,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens


In [22]:
yesno = ['sd_pdcontact', 'closed_approach', 'closed_outcome', 'closed_pdcontact']

In [23]:
yesno_cols(ambass, yesno)

In [24]:
format_datetime(ambass, 'encounter_timestamp')
format_datetime(ambass, 'encounter_datetime')

In [25]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_comments,closed_amenity,closed_patroncount,closed_comments,borough,sd_pdcontact,closed_approach,closed_outcome,closed_pdcontact
1,06-05-2020 19:03:54,06-05-2020 16:23:00,Prospect Park,NaN,OATH,Alex,Kolodka,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Brooklyn,NaN,NaN,NaN,NaN
2,06-05-2020 17:51:16,06-05-2020 17:30:00,Kissena Park,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,Open field/multi-purpose play area,NaN,NaN,NaN,NaN,Queens,0.0,NaN,NaN,NaN
3,06-05-2020 17:30:34,06-05-2020 17:30:00,Kissena Park,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,Open field/multi-purpose play area,NaN,NaN,NaN,NaN,Queens,0.0,NaN,NaN,NaN
4,06-05-2020 17:29:05,06-05-2020 14:45:00,Kissena Park,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Queens,NaN,NaN,NaN,NaN
5,06-05-2020 17:03:29,05-05-2020 16:00:00,Flushing Fields,NaN,OATH,Emmanuelasia,Vaught,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Queens,NaN,NaN,NaN,NaN


In [26]:
#Remove rows with no timestamp because these rows have no data
ambass = ambass[ambass['encounter_timestamp'].notnull()]

In [27]:
ambass = ambass.merge(site_ref, how = 'left', on = ['site_desc', 'borough'])[sql_cols]

In [28]:
#ambass[ambass['site_id'].isnull()]['site_desc'].unique()

In [29]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
0,06-05-2020 19:03:54,06-05-2020 16:23:00,B073,NaN,OATH,Alex,Kolodka,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn
1,06-05-2020 17:51:16,06-05-2020 17:30:00,Q024,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,Open field/multi-purpose play area,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
2,06-05-2020 17:30:34,06-05-2020 17:30:00,Q024,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,Open field/multi-purpose play area,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
3,06-05-2020 17:29:05,06-05-2020 14:45:00,Q024,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
4,06-05-2020 17:03:29,05-05-2020 16:00:00,Q086,NaN,OATH,Emmanuelasia,Vaught,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens


In [30]:
hash_rows(ambass, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Find the deltas based on the row hashes

In [31]:
ambass_deltas = (check_deltas(new_df = ambass, old_df = ambass_sql, on = ['encounter_timestamp', 'site_id'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [32]:
ambass_deltas.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough,dml_verb
0,06-05-2020 19:03:54,06-05-2020 16:23:00,B073,NaN,OATH,Alex,Kolodka,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn,I
1,06-05-2020 17:51:16,06-05-2020 17:30:00,Q024,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens,I
2,06-05-2020 17:30:34,06-05-2020 17:30:00,Q024,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens,I
3,06-05-2020 17:29:05,06-05-2020 14:45:00,Q024,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens,I
4,06-05-2020 17:03:29,05-05-2020 16:00:00,Q086,NaN,OATH,Emmanuelasia,Vaught,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens,I


In [33]:
ambass_inserts = ambass_deltas[ambass_deltas['dml_verb'] == 'I'][sql_cols]

In [34]:
ambass_inserts.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
0,06-05-2020 19:03:54,06-05-2020 16:23:00,B073,NaN,OATH,Alex,Kolodka,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn
1,06-05-2020 17:51:16,06-05-2020 17:30:00,Q024,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,Open field/multi-purpose play area,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
2,06-05-2020 17:30:34,06-05-2020 17:30:00,Q024,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,Open field/multi-purpose play area,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
3,06-05-2020 17:29:05,06-05-2020 14:45:00,Q024,NaN,DOC,Chris,Hennessey,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
4,06-05-2020 17:03:29,05-05-2020 16:00:00,Q086,NaN,OATH,Emmanuelasia,Vaught,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens


In [35]:
ambass_inserts.to_sql('tbl_cw_ambassador', engine, index = False, if_exists = 'append')

In [36]:
ambass_updates = ambass_deltas[ambass_deltas['dml_verb'] == 'U'][sql_cols]

In [37]:
ambass_updates.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
482,05-31-2020 16:56:32,05-31-2020 16:54:00,Q102,Jockey field,DEP,Riquelmer,Salinas,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
483,05-31-2020 16:56:32,05-31-2020 16:54:00,Q102,Jockey field,DEP,Riquelmer,Salinas,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1836,05-25-2020 20:17:06,05-25-2020 15:00:00,Q005-06,NaN,DEP,Joely,Delarosa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1837,05-25-2020 20:17:06,05-25-2020 15:00:00,Q005-06,NaN,DEP,Joely,Delarosa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
2425,05-25-2020 12:37:19,05-24-2020 12:00:00,Q021,Union Tpke and Francis Lewis Blvd,DEP,James,Tangredi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens


In [39]:
sql_update(ambass_updates, 'tbl_cw_ambassador', engine, ['encounter_timestamp', 'site_id'])